In [59]:
import pandas as pd
import numpy as np
from collections import Counter

In [63]:
b2b_df = pd.read_csv("data/B2B_Partner.csv")
complaints_df = pd.read_csv("data/complaints.csv")
player_df = pd.read_csv("data/Player_data.csv")
billing_df = pd.read_csv("data/Billing.csv")
contracts_df = pd.read_csv("data/contracts.csv")
users_df = pd.read_csv("data/User_subscription.csv")

In [67]:
def get_most_common_dev(devlist):
    c = Counter(devlist)
    return c.most_common(1)[0][0]

In [68]:
agg_dict = {
    "Duration":"mean",
    "DeviceId": pd.Series.nunique,
    "Hardware": get_most_common_dev,
    
}


player_agged = player_df.groupby("UserId").agg(agg_dict)

In [69]:
player_agged.reset_index(inplace = True)

In [70]:
player_agged.rename(columns = {"Duration":"avg_dur", 
                               "DeviceId":"num_devs", 
                               "Hardware":"freq_used_dev"}, inplace = True)

In [71]:
users_players_joined = users_df.merge(player_agged, left_on="profile_ID", right_on = "UserId", how = "left")

In [75]:
agg_dict = {
    "partner id":pd.Series.nunique,
    "device id": pd.Series.nunique,
    "device type": get_most_common_dev,
    
}


b2b_agged = b2b_df.groupby("customerID").agg(agg_dict)

In [76]:
b2b_agged.reset_index(inplace = True)

In [85]:
b2b_agged.rename(columns = {"partner id":"part_id", 
                            "deviceId":"num_devs", 
                            "device type":"freq_used_type"}, inplace = True)

In [86]:
users_b2b_joined = users_df.merge(b2b_agged, on = "customerID", how = "left")

In [87]:
users_players_joined

,profile_ID,Contact_type,DOB,Email,First_Name,Gender,Mobile_Number,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,customerID,Zipcode,UserId,avg_dur,num_devs,freq_used_dev
0,rsanchez,Email,6/10/2005,ldixson3h@engadget.com,Leese,Female,391-447-5117,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,2546-KZAAT,665 33,rsanchez,1512.277778,18,Android Phone
1,karina10,Email,1/12/2005,rearngy40@sphinn.com,Rubetta,Female,777-570-1291,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,2371-KFUOG,111721,karina10,1329.500000,8,iPad
2,sethconrad,Email,10/30/1992,tbecom72@spotify.com,Tait,Male,901-892-5768,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,0188-GWFLE,NaN,sethconrad,1478.166667,18,PC/Laptop
3,ewilson,Email,8/14/2015,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,0289-IVARM,NaN,ewilson,1118.133333,15,PC/Laptop
4,evanvelasquez,Email,5/25/1992,dmencib1@cisco.com,Devan,Female,902-541-3157,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,7672-VFMXZ,NaN,evanvelasquez,1557.428571,21,FireTV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,WhatsApp,12/14/2021,mcokee0@psu.edu,Maryjane,Female,755-848-1351,432660,3/22/2022,INACTIVE,NO,12/26/2022,India,Arunachal Pradesh,7361-YPXFS,NaN,brandili,1266.307692,13,PC/Laptop
996,davidlowe,WhatsApp,4/1/1993,ctothex@google.com.hk,Chic,Male,472-131-0900,432693,9/25/2021,INACTIVE,YES,11/14/2022,India,Arunachal Pradesh,3466-BYAVD,NaN,davidlowe,1199.000000,18,FireTV
997,carrierivera,WhatsApp,9/30/2002,bmacevillyh3@prlog.org,Bili,Female,568-875-8493,432771,11/24/2021,INACTIVE,YES,12/3/2022,India,Arunachal Pradesh,5652-MSDEY,6340,carrierivera,1482.000000,17,FireTV
998,ronniecoffey,WhatsApp,8/29/2002,lberrowhy@smh.com.au,Lilli,Female,380-420-0034,432802,1/21/2022,ACTIVE,NO,12/16/2022,India,Arunachal Pradesh,9970-QBCDA,NaN,ronniecoffey,1518.000000,30,PC/Laptop


In [90]:
billing_df["Customer Id"] = users_players_joined["customerID"]

In [93]:
users_training = users_players_joined.merge(billing_df, left_on = 'customerID', right_on = "Customer Id", how = 'left')

In [95]:
users_training.columns

Index(['profile_ID', 'Contact_type', 'DOB', 'Email', 'First_Name', 'Gender',
       'Mobile_Number', 'Pack_ID', 'Start_Date', 'Status', 'Upgradable',
       'Validity_End', 'Country', 'State', 'customerID', 'Zipcode', 'UserId',
       'avg_dur', 'num_devs', 'freq_used_dev', 'Customer Id', 'Billing Id',
       'Subscription Type ', 'Billing Type ', 'Customer Longevity'],
      dtype='object')

In [96]:
colums_needed = [
    'profile_ID',
    'DOB',
    'Contact_type',
    'Gender',
    'Pack_ID',
    'Start_Date',
    'Status',
    'Upgradable',
    'Validity_End',
    'Country',
    'State',
    'avg_dur', 
    'num_devs', 
    'freq_used_dev',
    'Subscription Type ', 
    'Billing Type ',
    'Customer Longevity'
]

In [98]:
train_df = users_training[colums_needed]
train_df.columns = [c.strip() for c in train_df.columns]

In [101]:
train_df["Subscription Type"] = train_df["Subscription Type"].str.replace(" Months", "")

/home/kezia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
train_df["Customer Longevity"] = train_df["Customer Longevity"].str.replace(" Years", "")

/home/kezia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
train_df.to_csv("data/training_data.csv", index = False)

In [106]:
train_df

,profile_ID,DOB,Contact_type,Gender,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,avg_dur,num_devs,freq_used_dev,Subscription Type,Billing Type,Customer Longevity
0,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,1512.277778,18,Android Phone,12,Paytm,4
1,karina10,1/12/2005,Email,Female,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,1329.500000,8,iPad,3,NetBanking,2
2,sethconrad,10/30/1992,Email,Male,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,1478.166667,18,PC/Laptop,6,NetBanking,3
3,ewilson,8/14/2015,Email,Female,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,1118.133333,15,PC/Laptop,12,Paytm,4
4,evanvelasquez,5/25/1992,Email,Female,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,1557.428571,21,FireTV,12,Paytm,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,12/14/2021,WhatsApp,Female,432660,3/22/2022,INACTIVE,NO,12/26/2022,India,Arunachal Pradesh,1266.307692,13,PC/Laptop,12,Paytm,4
996,davidlowe,4/1/1993,WhatsApp,Male,432693,9/25/2021,INACTIVE,YES,11/14/2022,India,Arunachal Pradesh,1199.000000,18,FireTV,12,Paytm,4
997,carrierivera,9/30/2002,WhatsApp,Female,432771,11/24/2021,INACTIVE,YES,12/3/2022,India,Arunachal Pradesh,1482.000000,17,FireTV,12,Paytm,4
998,ronniecoffey,8/29/2002,WhatsApp,Female,432802,1/21/2022,ACTIVE,NO,12/16/2022,India,Arunachal Pradesh,1518.000000,30,PC/Laptop,12,Paytm,4


In [110]:
churn_df = pd.DataFrame()
churn_df["profile_ID"] = train_df["profile_ID"]
churn_df["churn"] = np.random.choice([0,1], size = len(churn_df), p = [0.8,0.2])

In [111]:
churn_df.to_csv("data/churn.csv", index = False)